**This notebook shows how to run the inference in the training-time two-view settings on the validation or training set of MegaDepth to visualize the training metrics and losses.**

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from omegaconf import OmegaConf

from pixloc import run_Aachen
from pixloc.pixlib.datasets.megadepth import MegaDepth
from pixloc.pixlib.utils.tensor import batch_to_device, map_tensor
from pixloc.pixlib.utils.tools import set_seed
from pixloc.pixlib.utils.experiments import load_experiment
from pixloc.visualization.viz_2d import (
    plot_images, plot_keypoints, plot_matches, cm_RdGn,
    features_to_RGB, add_text)

torch.set_grad_enabled(False);
mpl.rcParams['image.interpolation'] = 'bilinear'

# Create a validation or training dataloader

In [5]:
conf = {
    'min_overlap': 0.4,
    'max_overlap': 1.0,
    'max_num_points3D': 512,
    'force_num_points3D': True,
    
    'resize': 512,
    'resize_by': 'min',
    'crop': 512,
    'optimal_crop': True,
    
    'init_pose': [0.75, 1.],
#     'init_pose': 'max_error',
#     'init_pose_max_error': 4,
#     'init_pose_num_samples': 50,
    
    'batch_size': 1,
    'seed': 1,
    'num_workers': 0,
}
loader = MegaDepth(conf).get_data_loader('val', shuffle=True)
orig_items = loader.dataset.items

[09/24/2021 16:58:00 pixloc.pixlib.datasets.base_dataset INFO] Creating dataset MegaDepth
[09/24/2021 16:58:00 pixloc.pixlib.datasets.megadepth INFO] Sampling new images or pairs with seed 1
100%|████████████████████████████████████████████████████| 77/77 [00:06<00:00, 11.66it/s]


# Load the training experiment

In [ ]:
# Name of the example experiment. Replace with your own training experiment.
exp = run_Aachen.experiment
device = 'cuda'
conf = {
    'optimizer': {'num_iters': 20,},
}
refiner = load_experiment(exp, conf).to(device)
print(OmegaConf.to_yaml(refiner.conf))

# Run on a few examples

- Reference image: red/green = reprojections of 3D points not/visible in the query at the ground truth pose
- Query image: red/blue/green = reprojections of 3D points at the initial/final/GT poses
- ΔP/ΔR/Δt are final errors in terms of 2D reprojections, rotation, and translation

In [ ]:
set_seed(7)
for _, data in zip(range(5), loader):
    data_ = batch_to_device(data, device)
    pred_ = refiner(data_)
    pred = map_tensor(pred_, lambda x: x[0].cpu())
    data = map_tensor(data, lambda x: x[0].cpu())
    cam_q = data['query']['camera']
    p3D_r = data['ref']['points3D']
        
    p2D_r, valid_r = data['ref']['camera'].world2image(p3D_r)
    p2D_q_gt, valid_q = cam_q.world2image(data['T_r2q_gt'] * p3D_r)
    p2D_q_init, _ = cam_q.world2image(data['T_r2q_init'] * p3D_r)
    p2D_q_opt, _ = cam_q.world2image(pred['T_r2q_opt'][-1] * p3D_r)
    valid = valid_q & valid_r
    
    losses = refiner.loss(pred_, data_)
    mets = refiner.metrics(pred_, data_)
    errP = f"ΔP {losses['reprojection_error/init'].item():.2f} -> {losses['reprojection_error'].item():.3f} px; "
    errR = f"ΔR {mets['R_error/init'].item():.2f} -> {mets['R_error'].item():.3f} deg; "
    errt = f"Δt {mets['t_error/init'].item():.2f} -> {mets['t_error'].item():.3f} %m"
    print(errP, errR, errt)

    imr, imq = data['ref']['image'].permute(1, 2, 0), data['query']['image'].permute(1, 2, 0)
    plot_images([imr, imq],titles=[(data['scene'][0], valid_r.sum().item(), valid_q.sum().item()), errP+'; '+errR])
    plot_keypoints([p2D_r[valid_r], p2D_q_gt[valid]], colors=[cm_RdGn(valid[valid_r]), 'lime'])
    plot_keypoints([np.empty((0, 2)), p2D_q_init[valid]], colors='red')
    plot_keypoints([np.empty((0, 2)), p2D_q_opt[valid]], colors='blue')
    add_text(0, 'reference')
    add_text(1, 'query')

    continue
    for i, (F0, F1) in enumerate(zip(pred['ref']['feature_maps'], pred['query']['feature_maps'])):
        C_r, C_q = pred['ref']['confidences'][i][0], pred['query']['confidences'][i][0]
        plot_images([C_r, C_q], cmaps=mpl.cm.turbo)
        add_text(0, f'Level {i}')
            
        axes = plt.gcf().axes
        axes[0].imshow(imr, alpha=0.2, extent=axes[0].images[0]._extent)
        axes[1].imshow(imq, alpha=0.2, extent=axes[1].images[0]._extent)
        plot_images(features_to_RGB(F0.numpy(), F1.numpy(), skip=1))